# **Eexploring major social media stock prices 2012-2022**  Data Visualization



#facebook, twitter, snapchat, etsy, pinterest daily stock prices from 2012

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import math
from bokeh.plotting import figure,output_notebook,show
from bokeh.layouts import gridplot,column
from bokeh.models import CrosshairTool
from bokeh.models import HoverTool
from bokeh.models import DatetimeTickFormatter
from bokeh.palettes import cividis
from ipywidgets import interact, SelectMultiple, IntRangeSlider
from bokeh.io import push_notebook
output_notebook()

In [ ]:
uploaded = files.upload()

Saving social media stocks 2012-2022.csv to social media stocks 2012-2022.csv


In [ ]:
df = pd.read_csv('social media stocks 2012-2022.csv')

In [ ]:
df.head(5)


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2012-05-18,FB,38.230000,38.230000,45.000000,38.000000,42.049999,573576400.0
1,2012-05-21,FB,34.029999,34.029999,36.660000,33.000000,36.529999,168192700.0
2,2012-05-22,FB,31.000000,31.000000,33.590000,30.940001,32.610001,101786600.0
3,2012-05-23,FB,32.000000,32.000000,32.500000,31.360001,31.370001,73600000.0
4,2012-05-24,FB,33.029999,33.029999,33.209999,31.770000,32.950001,50237200.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8398 entries, 0 to 8397
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       8398 non-null   object 
 1   Symbol     8398 non-null   object 
 2   Adj Close  8398 non-null   float64
 3   Close      8398 non-null   float64
 4   High       8398 non-null   float64
 5   Low        8398 non-null   float64
 6   Open       8398 non-null   float64
 7   Volume     8398 non-null   float64
dtypes: float64(6), object(2)
memory usage: 525.0+ KB


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# Droping "Adj Close" column

In [ ]:
set(df['Adj Close']==df['Close'])

{True}

In [ ]:
df.drop(columns='Adj Close',inplace=True)

In [ ]:
df['Symbol'].value_counts()

Symbol
FB      2488
TWTR    2118
ETSY    1758
SNAP    1285
PINS     749
Name: count, dtype: int64

# In order to analyze each stock separately, I use Groupby method to separate the datasets

In [ ]:
symbol_gr = df.groupby('Symbol')

In [ ]:
symbols = symbol_gr.groups.keys()

In [ ]:
symbols

dict_keys(['ETSY', 'FB', 'PINS', 'SNAP', 'TWTR'])

# For each Stock, we would like to explore and analyze the price chart and  volume bar chart accordingly during time range (2012 - 2022)
Time series

In [ ]:

for symbol in symbols:
  symbol_df = symbol_gr.get_group(symbol)
  date = symbol_df['Date']
  high = symbol_df['High']
  low = symbol_df['Low']
  close = symbol_df['Close']
  vol = symbol_df['Volume']
  hover = HoverTool(tooltips=[('Symbol',symbol),('Date',"@x{%F}"),('Price','@y')],formatters={'@x': 'datetime'})
  symbol_fig = figure(title=symbol+" "+"Stock: High, Low & Close Price".capitalize(),x_axis_label='Date',y_axis_label='Price',x_axis_type='datetime',tools=[hover],width=1000,height=600)
  symbol_fig.line(x=date,y=high,color='red',legend_label='high price'.capitalize(),name=symbol)
  symbol_fig.line(x=date,y=low,color='green',legend_label='low price'.capitalize(),name=symbol)
  symbol_fig.line(x=date,y=close,color='blue',legend_label='close price'.capitalize(),name=symbol)
  symbol_fig.add_tools(CrosshairTool())
  symbol_fig.legend.click_policy='hide'
  symbol_fig.legend.location ='top_left'
  vol_fig = figure(title=symbol+" "+'Volume',x_axis_label='Date',y_axis_label='Volume',x_axis_type='datetime',height=230,width=1000)
  vol_fig.vbar(x=date,top=vol,color='maroon',width=1,legend_label=symbol+" "+"Volume")
  vol_fig.left[0].formatter.use_scientific = False
  vol_fig.legend.location ='top_left'
  show(column(symbol_fig,vol_fig))
  print("-" * 150)



------------------------------------------------------------------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------------------------------------------------------------------


# Here, using time series and multiple select widget we want to see the growth perecentage of the price of each stock compared to the first close price in our time range



In [ ]:
palette = cividis(len(symbols))
symbol_lines={}
symbol_fig = figure(title='Percentage of Growth',x_axis_label='Date',y_axis_label='Percent',x_axis_type='datetime',tools=[hover],height=600,width=1000)
for i,symbol in enumerate(symbols):
  hover = HoverTool(tooltips=[('Symbol','$name'),('Date',"@x{%F}"),('Growth','@y %')],formatters={'@x': 'datetime'})

  symbol_df = symbol_gr.get_group(symbol)
  first = symbol_df['Close'].iloc[0]
  date = symbol_df['Date']
  percentage = ((symbol_df['Close']-first)/first) * 100
  line = symbol_fig.line(x=date,y=percentage,color=palette[i],legend_label=symbol,name=symbol)
  symbol_lines[symbol]=line
for symbol_line in symbol_lines.values():
  symbol_line.visible=False

symbol_fig.add_tools(CrosshairTool())
symbol_fig.legend.location='top_left'

handle = show(symbol_fig, notebook_handle=True)
selected_symbols = SelectMultiple (
    options = symbol_lines.keys()
)

@interact(symbols=selected_symbols)
def update(symbols):
  for symbol_name,symbol_line in symbol_lines.items():
    if symbol_name in symbols:
      symbol_line.visible = True
    else:
      symbol_line.visible = False
  push_notebook(handle=handle)


interactive(children=(SelectMultiple(description='symbols', options=('ETSY', 'FB', 'PINS', 'SNAP', 'TWTR'), va…